In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from nn_utils import Net, DEVICE, TRAINLOADER, train_nn, test_nn, freeze_parameters

torch.cuda.empty_cache()

PATH = './nn-models/cifar10-nn-model'

# load the pretrained NN model
net = Net()
net.load_state_dict(torch.load(PATH))
net.to(device=DEVICE)

test_nn(net=net, verbose=True)

GA

In [ ]:
import array
import random
import json
import numpy as np
from deap import base
from deap.benchmarks.tools import diversity, convergence, hypervolume
from deap import creator
from deap import tools
import matplotlib.pyplot as plt

freeze_parameters(net=net)

# store all the training dataset in a single batch
ALL_IMAGES = []
ALL_LABELS = []
for mini_batch in TRAINLOADER:
    ALL_IMAGES.append(mini_batch[0])
    ALL_LABELS.append(mini_batch[1])

ALL_IMAGES = torch.cat(ALL_IMAGES)
ALL_LABELS = torch.cat(ALL_LABELS)

# count the number of dimensions of the last layer
N_DIMENSION = 0
for param in net.out.parameters():
    N_DIMENSION += param.numel()

LOW_BOUND = -0.1
HIGH_BOUND = 0.3
N_BITS = 8
N_GENERATIONS = 100
MU = 200
CX_PB = 0.9
UNIFORM_CX_PB = 0.5
MUTATE_PB = 0.1
MUTATE_FLIP_PB = 1.0 / (N_DIMENSION * N_BITS)
ELITISM = 5

def decode(individual):
    real_numbers = []
    for i in range(N_DIMENSION):
        chromosome = individual[i*N_BITS:(i+1)*N_BITS]
        bit_string = ''.join(map(str, chromosome))
        num_as_int = int(bit_string, 2) # convert to int from base 2 list
        num_in_range = LOW_BOUND + (HIGH_BOUND - LOW_BOUND) * num_as_int / 2**N_BITS
        real_numbers.append(num_in_range)

    return real_numbers

def calculate_fitness(individual):
    # put the parameters into the neural network
    parameters = decode(individual=individual)
    parameters = torch.as_tensor(parameters, dtype=torch.float32, device=DEVICE)

    net.out.weight = torch.nn.Parameter(data=parameters[0:5120].reshape(10, 512))
    net.out.bias = torch.nn.Parameter(data=parameters[5120:5130])

    # go over the dataset once
    with torch.no_grad():
        # get a mini-batch from the training dataset
        images = ALL_IMAGES[0:1000].to(device=DEVICE)
        labels = ALL_LABELS[0:1000].to(device=DEVICE)

        preds = net(images) # forward mini-batch
        loss = F.cross_entropy(preds, labels) # calculate loss

    return (1 / 0.1 + loss.item()),

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, N_BITS*N_DIMENSION)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", calculate_fitness)
toolbox.register("crossover", tools.cxUniform, indpb=UNIFORM_CX_PB)
toolbox.register("mutate", tools.mutFlipBit, indpb=MUTATE_FLIP_PB)
toolbox.register("select", tools.selTournament, fit_attr='fitness')

def ga():
    # generate initial random population of individuals (parameters)
    pop = toolbox.population(n=MU)

    # evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    # track the performance of each generation
    best_individuals = [] 
    worst_individuals = [] 

    # begin the generational process
    for gen in range(1, N_GENERATIONS):

        # select the next generation individuals
        offspring = tools.selBest(pop, ELITISM) + toolbox.select(pop, len(pop)-ELITISM, 2)
        # clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
        
        # crossover make pairs of all (even, odd) in offspring
        for ind1, ind2 in zip(offspring[4::2], offspring[5::2]):
            if random.random() <= CX_PB:
                toolbox.crossover(ind1, ind2)
                del ind1.fitness.values
                del ind2.fitness.values

        # mutation
        for mutant in offspring:
            if random.random() <= MUTATE_PB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        # population is entirely replaced by the offspring
        pop[:] = offspring

        # track generation
        print(f'==== Generation {gen} finished ====')
        fits = [ind.fitness.values[0] for ind in pop]
        best_individuals.append(min(fits))
        worst_individuals.append(max(fits))

    return pop, best_individuals, worst_individuals
        
pop, best_individuals, worst_individuals = ga()

In [ ]:
best_individual = tools.selBest(pop, 1)[0]

plt.plot(np.array(best_individuals), color='b', linestyle='dashed', marker='.', linewidth=0.5, label='Worst individual')
plt.plot(np.array(worst_individuals), color='r', linestyle='dashed', marker='.', linewidth=0.5, label='Best individual')
plt.ylabel('Fitness value (Loss function')
plt.xlabel('Generation')
plt.show()

In [ ]:
# put best parameters back into the neural network
parameters = torch.as_tensor(best_individual, dtype=torch.float32, device=DEVICE)
net.out.weight = torch.nn.Parameter(data=parameters[0:5120].reshape(10, 512))
net.out.bias = torch.nn.Parameter(data=parameters[5120:5130])

print(net.out.weight.shape)
print(net.out.bias.shape)

# test the neural network
test_nn(net=net, verbose=True)